<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parte 1 - Solicitação de dados primários ao usuário

In [11]:
# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO #

# Pedir o nome da cidade e o total de resíduos produzidos por kg/mês na cidade
cidade = input('Informe o nome da cidade: ')
residuos_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))

# Pedir ao usuário a quantidade de bairros
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

bairros = []  # Lista para armazenar os nomes dos bairros
habitantes_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Pedir ao usuário os nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    bairros.append(bairro)

# Pedir ao usuário o número de habitantes de cada bairro e validar se é um número inteiro válido
for bairro in bairros:
    while True:
        try:
            habitantes = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
            if habitantes < 0:
                print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
                continue
            else:
                habitantes_bairro.append(habitantes)
                break
        except ValueError:
            print('Por favor, insira um número inteiro para o número de habitantes.')

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA #

total_habitantes = sum(habitantes_bairro)

residuos_por_bairro_semanal = []

for i, habitantes in enumerate(habitantes_bairro):
    proporcao = habitantes / total_habitantes
    residuos_bairro_semanal = round((proporcao * residuos_cidade) / 4)  # Dividindo por 4 para obter a quantidade semanal
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

# PARTE 3 - DIMENSIONAMENTO DAS ESTAÇÕES DE COLETA POR BAIRRO #

# Fator de conversão: quantos metros quadrados cada kg de resíduo ocupa
fator_area_armaz_por_kg = 2

# Lista para armazenar o número de estações de coleta por bairro
num_estacoes_por_bairro = []

# Altura mínima definida para depósitos e setor de triagem (em metros)
altura_minima = 4.0  # 4 metros

# Limite máximo de área para a Estação de Coleta por bairro
area_maxima_estacao = 1500.0  # 1500 metros quadrados

# Itera sobre cada bairro para calcular o número de estações de coleta
for bairro, residuos_diarios in zip(bairros, residuos_por_bairro_semanal):
    # Calcula a área necessária para o armazenamento de resíduos para as estações por bairro com base na quantidade de resíduos semanais
    area_necessaria_armaz_bairro = residuos_diarios * fator_area_armaz_por_kg

    # Calcula a área de logística adicional (15% da área necessária)
    area_de_logistica = area_necessaria_armaz_bairro * 0.15
    area_total_com_logistica = area_necessaria_armaz_bairro + area_de_logistica

    # Dimensionamento do depósito (retângulo) do bairro atual
    altura_armaz = altura_minima
    largura_armaz = area_total_com_logistica / altura_armaz
    area_real_armaz = largura_armaz * altura_armaz

    # Ajusta as dimensões do retângulo até alcançar a área necessária para o bairro
    while area_real_armaz < area_total_com_logistica:
        altura_armaz += 1
        largura_armaz = area_total_com_logistica / altura_armaz
        area_real_armaz = largura_armaz * altura_armaz

    # Calcula o perímetro do depósito (retângulo) do bairro
    perimetro_armaz = 2 * (largura_armaz + altura_armaz)

    # Dimensionamento do setor de triagem (retângulo) do bairro atual
    largura_triagem = 2 * largura_armaz  # Largura do setor de triagem é duas vezes a largura do depósito
    altura_triagem = altura_minima
    area_triagem = largura_triagem * altura_triagem

    # Calcula a área total da Estação de Coleta
    area_total_estacao = 2 * area_real_armaz + area_triagem

    # Verifica quantas estações de coleta são necessárias para não ultrapassar o limite máximo de área
    num_estacoes = area_total_estacao / area_maxima_estacao
    num_estacoes_por_bairro.append(num_estacoes)

# PARTE 4 - IMPRESSÃO DOS RESULTADOS POR BAIRRO #

print(f'CIDADE: {cidade}')
print('=-=' * 30)
print()

# Impressão da quantidade de resíduos gerados por semana em cada bairro
print('QUANTIDADE DE RESÍDUOS GERADOS POR SEMANA POR BAIRRO')
print()
for bairro, residuos_semanais in zip(bairros, residuos_por_bairro_semanal):
    print(f'No bairro {bairro}, a quantidade de resíduos por semana é de aproximadamente {residuos_semanais} kg.')
    print()

print('=-=' * 30)
print()

# Impressão do número de estações de coleta por bairro
print('NÚMERO DE ESTAÇÕES DE COLETA POR BAIRRO')
print()
for bairro, num_estacoes in zip(bairros, num_estacoes_por_bairro):
    print(f'No bairro {bairro}, são necessárias aproximadamente {int(num_estacoes)} estações de coleta.')
    print()

print('=-=' * 30)

Informe o nome da cidade: Natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 40000
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: Lagoa
Digite o nome do 2º bairro: Lagoinha
Digite o número de habitantes do bairro Lagoa: 400
Digite o número de habitantes do bairro Lagoinha: 300
CIDADE: Natal
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

QUANTIDADE DE RESÍDUOS GERADOS POR SEMANA POR BAIRRO

No bairro Lagoa, a quantidade de resíduos por semana é de aproximadamente 5714 kg.

No bairro Lagoinha, a quantidade de resíduos por semana é de aproximadamente 4286 kg.

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

NÚMERO DE ESTAÇÕES DE COLETA POR BAIRRO

No bairro Lagoa, são necessárias aproximadamente 35 estações de coleta.

No bairro Lagoinha, são necessárias aproximadamente 26 estações de coleta.

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

In [ ]:

#_____________________________________________________________________________________________________________________________________________________________________________

# PARTE 6 - QUANTOS CAMINHÕES SERÃO NECESSÁRIOS #                                                                                      (priorizar estruturas de controle)

import math

caminhão = 7000
Quantidade_de_caminhões_necessarios = math.ceil(residuos_semanais/caminhão)
print(f"Quantidade de caminhões necessários: {Quantidade_de_caminhões_necessarios}")

